In [14]:
from LIM_b7 import *

In [15]:
b_150_z0 = 0.32 # eq 21
alpha_90 = -1.5 # slope Ly continuum BUT NOT DETECTED
zpiv = 0.3
zpiv_1 = 1
zpiv_2 = 2

# parameter values: 
# _post = posteriors in table 1 
# _fid = fiducial under fig 3 

e_150_z0_post = pow(10,25.13) / b_150_z0
alpha_150_z0_post = -0.08
alpha_110_z0_post = -3.71

gamma_150_post = 2.06
CA_150_post = 1.85
CA_110_post = 0.50

EW_Lya_z03_post = -6.17
EW_Lya_z1_post = 88.02

fLy_c_z1_post = pow(10,-0.53)
fLy_c_z2_post = pow(10,-0.84)

par_post = [e_150_z0_post,alpha_150_z0_post,gamma_150_post,CA_150_post,alpha_110_z0_post,CA_110_post,EW_Lya_z03_post,EW_Lya_z1_post,fLy_c_z1_post,fLy_c_z2_post]

e_150_z0_fid = pow(10,25.) / b_150_z0
alpha_150_z0_fid = 0.
alpha_110_z0_fid = -3.

gamma_150_fid = 2.
CA_150_fid = 0.
CA_110_fid = 0.

EW_Lya_z1_fid = 0.
EW_Lya_z03_fid = 0.

fLy_c_z1_fid = pow(10,-5)
fLy_c_z2_fid = pow(10,-5)

par_fid = [e_150_z0_fid,alpha_150_z0_fid,gamma_150_fid,CA_150_fid,alpha_110_z0_fid,CA_110_fid,EW_Lya_z03_fid,EW_Lya_z1_fid,fLy_c_z1_fid,fLy_c_z2_fid]


In [16]:
nu_scale = lambda wavelenght: (cu.c / (wavelenght*u.nm.to(u.m)*u.m)).to(u.Hz)

In [70]:
def e_nu(nu_arr, z, which_par = 'fid'):
    
    e_150_z0, alpha_150_z0, gamma_150, CA_150, alpha_110_z0, CA_110, EW_Lya_z03, EW_Lya_z1, fLy_c_z1, fLy_c_z2 = par_fid if which_par == 'fid' else par_post if which_par == 'post' else []

    e_150 =  e_150_z0 *(1+z)*gamma_150 # continuum emissivity 
    alpha_150 = alpha_150_z0 + CA_150*np.log10(1+z)

    alpha_110 = alpha_110_z0 + CA_110*np.log10(1+z)

    CLya = (EW_Lya_z1 - EW_Lya_z03)/np.log10((1+zpiv_1)/(1+zpiv))
    EW_Lya = (CLya * np.log10((1+z)/(1+zpiv)) + EW_Lya_z03) # equivalent width, can be + (emission) or - (absorption)
    
    CLy_c = (np.log10(fLy_c_z2)-np.log10(fLy_c_z1))/np.log10((1+zpiv_2)/(1+zpiv_1))
    fLy_c = pow(10,CLy_c* np.log10((1+z)/(1+zpiv_1)) + np.log10(fLy_c_z2))# ionizing Ly continuum excape fraction

    e_nu_arr =  np.zeros(len(nu_arr))

    for i in range(len(nu_arr)):

        nu = nu_arr[i]
        if (cu.c / nu).to(u.nm) > (121.6)*u.nm :
        # only continuum 
            e_nu = e_150 * (nu / nu_scale(150))**alpha_150

        elif (cu.c / nu).to(u.nm) == (121.6)*u.nm :
        # line emission
            e_nu = e_150 * (nu_scale(121.6) / nu_scale(150.))**alpha_150 * ((nu / nu_scale(121.6))**alpha_110 +  EW_Lya*nu**2/cu.c)

        elif (cu.c / nu).to(u.nm) < (121.6)*u.nm and (cu.c / nu).to(u.nm) > (91.2)*u.nm :
        # continuum close to line
            e_nu = e_150 * (nu_scale(121.6) / nu_scale(150.))**alpha_150 * ((nu / nu_scale(121.6))**alpha_110)

        elif (cu.c / nu).to(u.nm) < (91.2)*u.nm :
        # Ly break
            e_nu = fLy_c*e_150*\
            (nu_scale(121.6)/ nu_scale(150.))**alpha_150 \
            *(nu_scale(91.2) / nu_scale(121.6))**alpha_110 \
            * (nu /nu_scale(91.2))**alpha_90


        e_nu_arr[i] = e_nu

    return e_nu_arr


In [76]:
CLya = (EW_Lya_z1_fid - EW_Lya_z03_fid)/np.log10((1+zpiv_1)/(1+zpiv))
EW_Lya = (CLya * np.log10((1+0)/(1+zpiv)) + EW_Lya_z03_fid)*(u.nm *1e-1)**-1 # equivalent width, can be + (emission) or - (absorption)
(EW_Lya*(5*u.Hz)**2/cu.c).to(u.Hz)


UnitConversionError: 'Hz2 s / (m nm)' (particle flux) and 'Hz' (frequency) are not convertible

In [71]:
wavelenght = np.concatenate((np.linspace(50,121.6),np.linspace(121.6,350)))

z = [0,0.3,0.6,1,1.5,2.1]
colors = ['b','teal','springgreen','orange','tomato','r']

plt.figure(figsize = (10,5))

nu_rest = nu_scale(wavelenght)
em_rest = e_nu(nu_rest,0.,'fid')

plt.subplot(211)
plt.plot(wavelenght, np.log10(em_rest))
plt.xlabel(r'$\lambda_{\rm rest}\,[{\rm nm}]$')
plt.ylabel(r'$\log_{10}\epsilon_{\nu_{\rm rest}}$')
plt.ylim(24,27)

plt.subplot(212)
for zv in z:
    em_obs = e_nu(nu_rest,zv,'fid')

    plt.plot(wavelenght, np.log10(em_obs),label=r'$z =%g$'%zv)
    plt.xlabel(r'$\lambda_{\rm obs}\,[{\rm nm}]$')
    plt.ylabel(r'$\log_{10}\epsilon_{\nu_{\rm obs}}$')

UnitConversionError: Can only apply 'add' function to quantities with compatible dimensions

<Figure size 720x360 with 0 Axes>

In [50]:
wavelenght

array([ 50.        ,  51.46122449,  52.92244898,  54.38367347,
        55.84489796,  57.30612245,  58.76734694,  60.22857143,
        61.68979592,  63.15102041,  64.6122449 ,  66.07346939,
        67.53469388,  68.99591837,  70.45714286,  71.91836735,
        73.37959184,  74.84081633,  76.30204082,  77.76326531,
        79.2244898 ,  80.68571429,  82.14693878,  83.60816327,
        85.06938776,  86.53061224,  87.99183673,  89.45306122,
        90.91428571,  92.3755102 ,  93.83673469,  95.29795918,
        96.75918367,  98.22040816,  99.68163265, 101.14285714,
       102.60408163, 104.06530612, 105.52653061, 106.9877551 ,
       108.44897959, 109.91020408, 111.37142857, 112.83265306,
       114.29387755, 115.75510204, 117.21632653, 118.67755102,
       120.13877551, 121.6       , 121.6       , 126.26122449,
       130.92244898, 135.58367347, 140.24489796, 144.90612245,
       149.56734694, 154.22857143, 158.88979592, 163.55102041,
       168.2122449 , 172.87346939, 177.53469388, 182.19